In [ ]:
import pandas as pandas
dataframe = pandas.read_csv("dataset.csv").dropna()
dataframe["date"] = pandas.to_datetime(dataframe['date'], format='%Y-%m-%d %H:%M:%S')
dataframe["datetimestamp"] = dataframe["date"]
dataframe["NSM"] = (dataframe["date"] - pandas.to_datetime(dataframe['date'].dt.date, format='%Y-%m-%d')).dt.total_seconds()
dataframe["week_status"] = dataframe["date"].apply(lambda date: 1 if(date.weekday() <= 4) else 0)
dataframe["day_of_week"] = dataframe["date"].dt.day_name()
dataframe.info()

In [ ]:
from pathlib import Path  
output_dataset_path = Path("./dataframeResult/modifiedDataset.csv")
output_dataset_path.parent.mkdir(parents=True, exist_ok=True)
dataframe.to_csv(Path(output_dataset_path))  
dataframe.head()

In [ ]:
dataframe.plot.line(
    x = "datetimestamp", 
    y = "Appliances", 
    figsize = (48,9), 
    title = "Appliances energy consumption vs datetimestamp", 
    ylabel = "Appliances energy consumption(in Wh)", 
    xlabel = "datetimestamp"
)

import datetime as datetime
dataframe.where((dataframe["datetimestamp"] - dataframe["datetimestamp"].min()) < datetime.timedelta(days = 7)).plot.line(
    x = "datetimestamp", 
    y = "Appliances", 
    figsize = (48,9), 
    title = "Appliances energy consumption vs datetimestamp", 
    ylabel = "Appliances energy consumption(in Wh)", 
    xlabel = "datetimestamp"
)

In [ ]:
# Fig: 8 not working
# # # # dataframe["Appliances"].value_counts().plot.hist(
# # # #     x = "Appliances", 
# # # #     y = "Frequency", 
# # # #     figsize = (48,9), 
# # # #     title = "Frquency vs Appliances energy consumption", 
# # # #     ylabel = "Frequency", 
# # # #     xlabel = "Appliances energy consumption(in Wh)"
# # # # )

In [ ]:
# # Not Happy with multiple Regression

# from pyspark.sql import SparkSession
# dataframe = SparkSession.builder.appName("Mining_Big_Data_Assignment_1").getOrCreate().read.csv("./dataframeResult/modifiedDataset.csv", header = True, inferSchema = True)
# dataframe.dropna()

# from pyspark.sql.types import DoubleType
# dataframe = dataframe.withColumn("Appliances", dataframe["Appliances"].cast(DoubleType()))

# from pyspark.ml.feature import VectorAssembler
# train_dataset, test_dataset = VectorAssembler(inputCols=["lights", "T1", "RH_1", "T2", "RH_2", "T3", "RH_3", "T4", "RH_4", "T5", "RH_5", "T6", "RH_6", "T7", "RH_7", "T8", "RH_8", "T9", "RH_9", "T_out", "Press_mm_hg", "RH_out", "Windspeed", "Visibility", "Tdewpoint", "rv1", "rv2", "NSM", "week_status"], outputCol="assembled_feature_vector").transform(dataframe).randomSplit([0.75, 0.25])

# from pyspark.ml.regression import Regress
# model = LinearRegression(featuresCol="assembled_feature_vector", labelCol="Appliances").fit(train_dataset).transform(test_dataset)

# from pyspark.ml.evaluation import RegressionEvaluator
# root_mean_square_error = RegressionEvaluator(labelCol="Appliances", predictionCol="Appliances", metricName="rmse").evaluate(model)

# print("Root Mean Square Error = " + str(root_mean_square_error))

# import matplotlib.pyplot as plot
# dataframe = model.toPandas()
# plot.scatter(dataframe['lights'], dataframe['Appliances'])
# plot.xlabel('Appliances energy consumption(in Wh)')
# plot.ylabel('Light energy consumption(in Wh)')
# plot.show()

# dataframe = pandas.read_csv("./dataframeResult/modifiedDataset.csv").dropna()
# plot.scatter(dataframe['Appliances'], dataframe['lights'])
# plot.plot(dataframe['lights'])
# plot.xlabel('Appliances energy consumption(in Wh)')
# plot.ylabel('Light energy consumption(in Wh)')
# plot.show()